In [2]:
from google.colab import drive
drive.mount('/content/drive')

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import datetime

!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf
plt.rc('font', family='NanumBarunGothic') 

MessageError: ignored

# csv 파일 불러오기

In [ ]:
cooking = pd.read_csv("/content/drive/MyDrive/Projectdata/Cooking_data_set.csv")
error = pd.read_csv("/content/drive/MyDrive/Projectdata/Error_Message.csv")
product = pd.read_csv("/content/drive/MyDrive/Projectdata/Product_Name.csv")
booking = pd.read_csv("/content/drive/MyDrive/Projectdata/booking_data.csv")

# 데이터 전처리

## NaN 제거
- 품목코드 3개 없는 것 삭제

In [ ]:
# NaN 제거
error_set = error.dropna()

## 발생시간, 종료시간, 오류조치시간

### 발생시간, 종료시간 datetime 형변환

In [ ]:
error_set['발생시간'] = pd.to_datetime(error_set['발생시간'], format='%H:%M')
error_set['종료시간'] = pd.to_datetime(error_set['종료시간'], format='%H:%M')

In [ ]:
error_set.dtypes

### 오류조치시간 재계산

In [ ]:
error_set_minus = error_set[pd.to_datetime(error_set['종료시간'], format='%H:%M') < pd.to_datetime(error_set['발생시간'], format='%H:%M')]
error_set_minus['종료시간'] = pd.to_datetime(error_set_minus['종료시간'], format='%H:%M') + datetime.timedelta(days=1)

In [ ]:
idx = error_set_minus.index

error_set.loc[idx,'종료시간'] = error_set_minus.loc[idx,'종료시간']

In [ ]:
error_set['오류조치시간'] = (pd.to_datetime(error_set['종료시간']).astype(np.int64) - pd.to_datetime(error_set['발생시간']).astype(np.int64)) // 60000000000

In [ ]:
error_set

## cooking과 error 합치기

In [ ]:
cooking

In [ ]:
# 생산시간이 없는 row만 가져온다.
cooking_notime = cooking[cooking['생산시간'].isnull()]
# cooking_notime['생산시간'] = cooking_notime['생산시간'].fillna(0)

# 생산시간 column 삭제 시도
cooking_notime = cooking_notime.drop(['생산시간'], axis=1)

In [ ]:
cooking_notime

In [ ]:
cooking_notime = cooking_notime.dropna()

In [ ]:
cooking_notime

In [ ]:
error_cooking = error_set.merge(cooking_notime, left_on=['순번','생산라인코드','생산라인명','생산일자','품목코드','품목명'],
right_on=['순번','생산라인코드','생산라인명','생산일자','품목코드','품목명'])
error_cooking

In [ ]:
error_cooking.isnull().sum()

In [ ]:
error_set.head()
#['순번','생산라인코드','생산라인명','생산일자','품목코드','품목명']

In [ ]:
cooking_notime.head()
# ['순번','생산라인코드','생산라인명','생산일자','품목코드','품목명']

In [ ]:
error_cooking_simple = error_cooking.copy()

error_cooking_simple['충전실온도'] = round(error_cooking['충전실온도'])
error_cooking_simple['실링온도'] = round(error_cooking['실링온도'])
error_cooking_simple['쿠킹온도'] = round(error_cooking['쿠킹온도'])
error_cooking_simple['쿠킹스팀압력'] = round(error_cooking['쿠킹스팀압력'])
error_cooking_simple['실링압력'] = round(error_cooking['실링압력'])

error_cooking_simple

In [ ]:
sns.barplot(data=error_cooking_simple, x='충전실온도')

In [ ]:
error_cooking['충전실온도'].min()

In [ ]:
t_charge = error_cooking_simple.groupby('충전실온도',as_index=False).size()
t_shilling = error_cooking_simple.groupby('실링온도',as_index=False).size()
t_cooking = error_cooking_simple.groupby('쿠킹온도',as_index=False).size()
p_cooking = error_cooking_simple.groupby('쿠킹스팀압력',as_index=False).size()
p_shilling = error_cooking_simple.groupby('실링압력',as_index=False).size()
p_shilling

In [ ]:
a = error_cooking_simple.groupby('충전실온도').count()
a

In [ ]:
plt.figure(figsize=(12,8))

plt.subplot(3,2,1)
sns.barplot(data=t_charge, x = '충전실온도', y='size')

plt.subplot(3,2,2)
sns.barplot(data=t_shilling, x = 'size', y='실링온도')

plt.subplot(3,2,3)
sns.barplot(data=t_cooking, x = '쿠킹온도', y='size')

plt.subplot(3,2,4)
sns.barplot(data=p_cooking, x = '쿠킹스팀압력', y='size')

plt.subplot(3,2,5)
sns.barplot(data=p_shilling, x = '실링압력', y='size')

plt.show()

In [ ]:
plt.figure(figsize=(12,8))

plt.subplot(3,2,1)
sns.countplot(data=error_cooking_simple, x = '충전실온도')

plt.subplot(3,2,2)
sns.countplot(data=error_cooking_simple, x = '실링온도')

plt.subplot(3,2,3)
sns.countplot(data=error_cooking_simple, x = '쿠킹온도')

plt.subplot(3,2,4)
sns.countplot(data=error_cooking_simple, x = '쿠킹스팀압력')

plt.subplot(3,2,5)
sns.countplot(data=error_cooking_simple, x = '실링압력')

plt.show()

In [ ]:
plt.figure(figsize=(12,8))

plt.subplot(3,2,1)
sns.histplot(data=error_cooking, x = '충전실온도', bins=6)

plt.subplot(3,2,2)
sns.histplot(data=error_cooking, x = '실링온도', bins=6)

plt.subplot(3,2,3)
sns.histplot(data=error_cooking, x = '쿠킹온도', bins=5)

plt.subplot(3,2,4)
sns.histplot(data=error_cooking, x = '쿠킹스팀압력', bins=5)

plt.subplot(3,2,5)
sns.histplot(data=error_cooking, x = '실링압력', bins=5)


plt.show()

In [ ]:
t_charge = error_cooking_simple.groupby('충전실온도',as_index=False).size()
t_shilling = error_cooking_simple.groupby('실링온도',as_index=False).size()
t_cooking = error_cooking_simple.groupby('쿠킹온도',as_index=False).size()
p_cooking = error_cooking_simple.groupby('쿠킹스팀압력',as_index=False).size()
p_shilling = error_cooking_simple.groupby('실링압력',as_index=False).size()

In [ ]:
plt.figure(figsize=(12,8))

plt.subplot(3,2,1)
sns.lineplot(data=t_charge, x = '충전실온도', y='size')

plt.subplot(3,2,2)
sns.lineplot(data=t_shilling, x = '실링온도', y='size')

plt.subplot(3,2,3)
sns.lineplot(data=t_cooking, x = '쿠킹온도', y='size')

plt.subplot(3,2,4)
sns.lineplot(data=p_cooking, x = '쿠킹스팀압력', y='size')

plt.subplot(3,2,5)
sns.lineplot(data=p_shilling, x = '실링압력', y='size')


plt.show()

In [ ]:
error_cooking_w005 = error_cooking[error_cooking['작업장코드'] == 'W005']
error_cooking_w005.head()

In [ ]:
plt.figure(figsize=(12,8))

plt.subplot(3,2,1)
sns.histplot(data=error_cooking_w005, x = '충전실온도', bins=6)

plt.subplot(3,2,2)
sns.histplot(data=error_cooking_w005, x = '실링온도', bins=6)

plt.subplot(3,2,3)
sns.histplot(data=error_cooking_w005, x = '쿠킹온도', bins=5)

plt.subplot(3,2,4)
sns.histplot(data=error_cooking_w005, x = '쿠킹스팀압력', bins=5)

plt.subplot(3,2,5)
sns.histplot(data=error_cooking_w005, x = '실링압력', bins=5)


plt.show()

In [ ]:
error_cooking[error_cooking['Error_Message'].str.contains('불량')]

In [ ]:
error_cooking.groupby('Error_Message')['충전실온도', '실링온도', '쿠킹온도', '쿠킹스팀압력', '실링압력'].mean()

In [ ]:
error_cooking

In [ ]:
cooking_sample= cooking.copy()

In [ ]:
cooking_sample = cooking_sample.drop(['생산시간'], axis=1)

In [ ]:
cooking_sample = cooking_sample.dropna()
cooking_sample

In [ ]:
## 전체데이터
plt.figure(figsize=(12,8))

plt.subplot(3,2,1)
sns.histplot(data=cooking_sample, x = '충전실온도', bins=6)

plt.subplot(3,2,2)
sns.histplot(data=cooking_sample, x = '실링온도', bins=6)

plt.subplot(3,2,3)
sns.histplot(data=cooking_sample, x = '쿠킹온도', bins=5)

plt.subplot(3,2,4)
sns.histplot(data=cooking_sample, x = '쿠킹스팀압력', bins=5)

plt.subplot(3,2,5)
sns.histplot(data=cooking_sample, x = '실링압력', bins=5)


plt.show()

In [ ]:
## 오류 
plt.figure(figsize=(12,8))

plt.subplot(3,2,1)
sns.histplot(data=error_cooking, x = '충전실온도', bins=6)

plt.subplot(3,2,2)
sns.histplot(data=error_cooking, x = '실링온도', bins=6)

plt.subplot(3,2,3)
sns.histplot(data=error_cooking, x = '쿠킹온도', bins=5)

plt.subplot(3,2,4)
sns.histplot(data=error_cooking, x = '쿠킹스팀압력', bins=5)

plt.subplot(3,2,5)
sns.histplot(data=error_cooking, x = '실링압력', bins=5)


plt.show()

In [ ]:
## 전체데이터
plt.figure(figsize=(12,8))

plt.subplot(2,5,1)
sns.histplot(data=cooking_sample, x = '충전실온도', bins=6)

plt.subplot(2,5,2)
sns.histplot(data=cooking_sample, x = '실링온도', bins=6)

plt.subplot(2,5,3)
sns.histplot(data=cooking_sample, x = '쿠킹온도', bins=5)

plt.subplot(2,5,4)
sns.histplot(data=cooking_sample, x = '쿠킹스팀압력', bins=5)

plt.subplot(2,5,5)
sns.histplot(data=cooking_sample, x = '실링압력', bins=5)

## 오류 

plt.subplot(2,5,1)
sns.histplot(data=error_cooking, x = '충전실온도', bins=6, color='r')
plt.xticks([70, 75])

plt.subplot(2,5,2)
sns.histplot(data=error_cooking, x = '실링온도', bins=6, color='r')
plt.xticks([70, 90, 115, 140])

plt.subplot(2,5,3)
sns.histplot(data=error_cooking, x = '쿠킹온도', bins=5, color='r')
plt.xticks([100, 130, 160, 190])

plt.subplot(2,5,4)
sns.histplot(data=error_cooking, x = '쿠킹스팀압력', bins=5, color='r')
plt.xticks([23,24, 25])

plt.subplot(2,5,5)
sns.histplot(data=error_cooking, x = '실링압력', bins=5, color='r')
plt.xticks([200, 210, 220])

plt.tight_layout()
plt.show()

In [ ]:
## 전체데이터
plt.figure(figsize	=(12,8))
sns.color_palette()

plt.subplot(2,5,1)
sns.boxplot(data=cooking_sample, x = '충전실온도', y='작업장코드')

plt.subplot(2,5,2)
sns.boxplot(data=cooking_sample, x = '실링온도', y='작업장코드')

plt.subplot(2,5,3)
sns.boxplot(data=cooking_sample, x = '쿠킹온도', y='작업장코드')

plt.subplot(2,5,4)
sns.boxplot(data=cooking_sample, x = '쿠킹스팀압력', y='작업장코드')

plt.subplot(2,5,5)
sns.boxplot(data=cooking_sample, x = '실링압력', y='작업장코드')

## 오류 

plt.subplot(2,5,1)
sns.boxplot(data=error_cooking, x = '충전실온도', y='작업장코드', boxprops=dict(alpha=0.6), color='y')
plt.xticks([70, 75])

plt.subplot(2,5,2)
sns.boxplot(data=error_cooking, x = '실링온도', y='작업장코드', boxprops=dict(alpha=0.6), color='y')
plt.xticks([70, 90, 115, 140])

plt.subplot(2,5,3)
sns.boxplot(data=error_cooking, x = '쿠킹온도', y='작업장코드', boxprops=dict(alpha=0.6), color='y')
plt.xticks([100, 130, 160, 190])

plt.subplot(2,5,4)
sns.boxplot(data=error_cooking, x = '쿠킹스팀압력', y='작업장코드', boxprops=dict(alpha=0.6), color='y')
plt.xticks([23,24, 25])

plt.subplot(2,5,5)
sns.boxplot(data=error_cooking, x = '실링압력', y='작업장코드', boxprops=dict(alpha=0.6), color='y')
plt.xticks([200, 210, 220])

plt.tight_layout()

plt.show()

In [ ]:
## 전체데이터
plt.figure(figsize	=(12,8))
sns.color_palette()

plt.subplot(2,5,1)
sns.boxplot(data=cooking_sample, x = '충전실온도', y='작업장코드')

plt.subplot(2,5,2)
sns.boxplot(data=cooking_sample, x = '실링온도', y='작업장코드')

plt.subplot(2,5,3)
sns.boxplot(data=cooking_sample, x = '쿠킹온도', y='작업장코드')

plt.subplot(2,5,4)
sns.boxplot(data=cooking_sample, x = '쿠킹스팀압력', y='작업장코드')

plt.subplot(2,5,5)
sns.boxplot(data=cooking_sample, x = '실링압력', y='작업장코드')

## 오류 

plt.subplot(2,5,1)
sns.boxplot(data=error_cooking, x = '충전실온도', y='작업장코드', color='y')
plt.xticks([70, 75])

plt.subplot(2,5,2)
sns.boxplot(data=error_cooking, x = '실링온도', y='작업장코드', color='y')
plt.xticks([70, 90, 115, 140])

plt.subplot(2,5,3)
sns.boxplot(data=error_cooking, x = '쿠킹온도', y='작업장코드', color='y')
plt.xticks([100, 130, 160, 190])

plt.subplot(2,5,4)
sns.boxplot(data=error_cooking, x = '쿠킹스팀압력', y='작업장코드', color='y')
plt.xticks([23,24, 25])

plt.subplot(2,5,5)
sns.boxplot(data=error_cooking, x = '실링압력', y='작업장코드', color='y')
plt.xticks([200, 210, 220])

plt.tight_layout()

plt.show()

In [ ]:
error_set.head()

In [ ]:
cooking_notime.head()

In [ ]:
error_set['작업장코드'] = error_set['비가동코드'].apply(lambda x : x[:4])
error_set

error_cooking = error_set.merge(cooking_notime, left_on=['순번','생산라인코드','생산라인명','생산일자','품목코드','품목명'],
right_on=['순번','생산라인코드','생산라인명','생산일자','품목코드','품목명'])

In [ ]:
error_cooking2 = error_set.merge(cooking_notime, left_on=['순번','생산라인코드','생산라인명','생산일자','품목코드','품목명','작업장코드'], right_on=['순번','생산라인코드','생산라인명','생산일자','품목코드','품목명','작업장코드'])

In [ ]:
error_cooking2.groupby('작업장코드')['오류발생작업장명'].count()

In [ ]:
print(error_set.dtypes)
print('-------------------------------------------------')
print(cooking_notime.dtypes)
print('-------------------------------------------------')
print(error_cooking.dtypes)


In [ ]:
error_cooking[error_cooking.duplicated(subset=None, keep='first')]

# 336,337,338이 공통 행이다. 그래서 336만 남기고 337,338을 지운다 
# 온도,압력이 달라서 공통 행이 생겼다. 그래서 지운다. (아마 사람이 잘못 입력한듯)

In [ ]:
error_cooking.loc[336]

In [ ]:
error_cooking.loc[339,:]

In [ ]:
error_cooking = error_cooking.drop([337,338])

In [ ]:
error_cooking

In [ ]:
## 전체데이터
plt.figure(figsize	=(12,8))
sns.color_palette()

plt.subplot(2,5,1)
sns.boxplot(data=cooking_sample, x = '충전실온도', y='작업장코드')

plt.subplot(2,5,2)
sns.boxplot(data=cooking_sample, x = '실링온도', y='작업장코드')

plt.subplot(2,5,3)
sns.boxplot(data=cooking_sample, x = '쿠킹온도', y='작업장코드')

plt.subplot(2,5,4)
sns.boxplot(data=cooking_sample, x = '쿠킹스팀압력', y='작업장코드')

plt.subplot(2,5,5)
sns.boxplot(data=cooking_sample, x = '실링압력', y='작업장코드')

## 오류 

plt.subplot(2,5,1)
sns.boxplot(data=error_cooking, x = '충전실온도', y='작업장코드', color='y')
plt.xticks([70, 75])

plt.subplot(2,5,2)
sns.boxplot(data=error_cooking, x = '실링온도', y='작업장코드', color='y')
plt.xticks([70, 90, 115, 140])

plt.subplot(2,5,3)
sns.boxplot(data=error_cooking, x = '쿠킹온도', y='작업장코드', color='y')
plt.xticks([100, 130, 160, 190])

plt.subplot(2,5,4)
sns.boxplot(data=error_cooking, x = '쿠킹스팀압력', y='작업장코드', color='y')
plt.xticks([23,24, 25])

plt.subplot(2,5,5)
sns.boxplot(data=error_cooking, x = '실링압력', y='작업장코드', color='y')
plt.xticks([200, 210, 220])

plt.tight_layout()

plt.show()

In [ ]:
error_cooking_model_int = error_cooking.drop(['순번','생산라인코드','생산라인명','생산일자','발생시간','종료시간','오류조치시간','비가동코드','Error_Message','품목코드','품목명','오류발생작업장명','쿠킹스팀압력','작업장코드'], axis=1)
error_cooking_model_int

In [ ]:
error_cooking_model_int['충전실온도'] = error_cooking_model['충전실온도'].astype({'충전실온도' : 'int'})
error_cooking_model_int['실링온도'] = error_cooking_model['실링온도'].astype({'실링온도' : 'int'})
error_cooking_model_int['쿠킹온도'] = error_cooking_model['쿠킹온도'].astype({'쿠킹온도' : 'int'})
error_cooking_model_int['실링압력'] = error_cooking_model['실링압력'].astype({'실링압력' : 'int'})
error_cooking_model_int.dtypes

In [ ]:
error_cooking_model_int

In [ ]:
error_cooking_model_int.dtypes

# 머신러닝

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier

x = error_cooking_model_int
y = error_cooking['쿠킹스팀압력']

In [ ]:
y = y.astype({'쿠킹스팀압력':'int'})

In [ ]:
x

In [ ]:
x_tn, x_te, y_tn, y_te = train_test_split(x,y, test_size=0.3, random_state=0) # 학습시킬 데이터, 테스트용 데이터 설정

In [ ]:
from sklearn.preprocessing import StandardScaler

# 스케일링 하는 곳
std_scale = StandardScaler()
std_scale.fit(x_tn)
x_tn_std = std_scale.transform(x_tn)
x_te_std = std_scale.transform(x_te)

In [ ]:
from sklearn.ensemble import RandomForestClassifier

# 모델생성 randomforest로 만들어줌
clf_rf = RandomForestClassifier(max_depth=7, random_state=0)

clf_rf.fit(x_tn_std,y_tn)

In [ ]:
# 예측값 class에 들어가는 예측값을 넣는다.
pred_rf = clf_rf.predict(x_te_std)
print(pred_rf)

In [ ]:
# 정확도 검사

from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_te, pred_rf)
print(accuracy)

In [ ]:
# PDF 451.p에 보면 예측을  잘했는지 보여주는 것 
# 왼쪽상단에서 오른쪽 하단 (대각선으로) 숫자가 나올수록 정확도가 높다는 뜻
from sklearn.metrics import confusion_matrix

conf_matrix = confusion_matrix(y_te, pred_rf)
print(conf_matrix)

In [ ]:
# PDF 454.p/ precision = 정밀도, recall = 재현율, f1-score

from sklearn.metrics import classification_report
class_report = classification_report(y_te, pred_rf)
print(class_report)

In [ ]:
y

In [ ]:
error_cooking_model = error_cooking.drop(['순번','생산라인코드','생산라인명','생산일자','발생시간','종료시간','오류조치시간','비가동코드','Error_Message','품목코드','품목명','오류발생작업장명','쿠킹스팀압력','작업장코드'], axis=1)
error_cooking_model